# **Revenue Optimization - Determining how much to sell and store at a given time a day**

## Explanations of the  Revenue-Based Optimizer

Forecasted Energy Prices: Time-series predictions of energy prices.


Forecasted Energy Demand: Predicted demand patterns.


Battery State of Charge (SoC): Initial and current state of the battery.


Energy Storage Capacity: Maximum and minimum energy storage.


Charging/Discharging Rates: Maximum allowable rates for charging and discharging.

## Constraints of optimizer

Energy balance: initial state and current charge of battery + energy charged - energy sold




Capacity: between 0 and the capacity of the battery, the initial state and charge of battery must be between 



Charging rate: energy charged cannot be greater than max charging rate 


Discharge rate: energy sold cannot be greater than max discharging rate 

In [49]:
import pandas as pd

# Load predictions and convert them into a list format in order to loop through them in the optimizer function 
price_forecast = pd.read_csv('Submission.csv')
system_price_values = price_forecast['System_Price'].values


In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value
import numpy as np
import pandas as pd
# Load predictions and convert them into a list format in order to loop through them in the optimizer function 
price_forecast = pd.read_csv('Submission.csv')
system_price_values = price_forecast['System_Price'].values

# Parameters
T = len(system_price_values)  # Number of time periods
battery_capacity = 100  # Maximum battery capacity in MWh
max_charging_rate = 50  # Maximum charging rate in MW
max_discharging_rate = 50  # Maximum discharging rate in MW
initial_soc = 50  # Initial state of charge in MWh
charging_efficiency = 0.9  # Charging efficiency
discharging_efficiency = 0.9  # Discharging efficiency
safety_margin = 5  # Safety margin for battery state of charge (MWh)

# -----------------------------
# HEURISTIC PRE-PROCESSING STEP
# -----------------------------
sorted_prices = sorted(system_price_values)
low_cutoff = np.percentile(system_price_values, 15)   # bottom 20% are "low-price" periods
high_cutoff = np.percentile(system_price_values, 95)  # top 20% are "high-price" periods

low_price_periods = [t for t, p in enumerate(system_price_values) if p <= low_cutoff]
high_price_periods = [t for t, p in enumerate(system_price_values) if p >= high_cutoff]

# These sets identify when charging or discharging might be beneficial. 
# We'll incorporate these sets into the objective function as gentle nudges.

# -----------------------------
# OPTIMIZATION MODEL
# -----------------------------
problem = LpProblem("Revenue_Maximization", LpMaximize)

# Decision Variables
SoC = [LpVariable(f"SoC_{t}", lowBound=0, upBound=battery_capacity) for t in range(T)]
energy_sold = [LpVariable(f"Energy_Sold_{t}", lowBound=0, upBound=max_discharging_rate) for t in range(T)]
energy_charged = [LpVariable(f"Energy_Charged_{t}", lowBound=0, upBound=max_charging_rate) for t in range(T)]
is_charging = [LpVariable(f"Is_Charging_{t}", cat="Binary") for t in range(T)]
is_discharging = [LpVariable(f"Is_Discharging_{t}", cat="Binary") for t in range(T)]

# -----------------------------
# OBJECTIVE FUNCTION
# -----------------------------
# Primary goal: maximize revenue from selling energy at given prices
obj = lpSum(system_price_values[t] * energy_sold[t] for t in range(T)) \
    - lpSum(system_price_values[t] * energy_charged[t] for t in range(T))

# If you still want small bonuses (not usually necessary), you can add them:
charging_bonus = 0.1
discharging_bonus = 0.1

obj += lpSum(charging_bonus * energy_charged[t] for t in low_price_periods)
obj += lpSum(discharging_bonus * energy_sold[t] for t in high_price_periods)

problem += obj, "Total_Profit"


# -----------------------------
# CONSTRAINTS
# -----------------------------
for t in range(T):
    # State of charge balance
    if t == 0:
        problem += SoC[t] == initial_soc + charging_efficiency * energy_charged[t] - energy_sold[t]
    else:
        problem += SoC[t] == SoC[t-1] + charging_efficiency * energy_charged[t] - energy_sold[t]

    # Battery capacity constraints with safety margin
    problem += SoC[t] >= safety_margin
    problem += SoC[t] <= battery_capacity - safety_margin

    # Link binary variables to charging/discharging
    problem += energy_charged[t] <= max_charging_rate * is_charging[t]
    problem += energy_sold[t] <= max_discharging_rate * is_discharging[t]

    # No simultaneous charging and discharging
    problem += is_charging[t] + is_discharging[t] <= 1

# Solve the problem
problem.solve()

# Extract results
optimal_energy_sold = [energy_sold[t].varValue for t in range(T)]
optimal_energy_charged = [energy_charged[t].varValue for t in range(T)]
optimal_soc = [SoC[t].varValue for t in range(T)]
optimal_is_charging = [is_charging[t].varValue for t in range(T)]
optimal_is_discharging = [is_discharging[t].varValue for t in range(T)]

# Create a DataFrame for results
results_df = pd.DataFrame({
    "Price": system_price_values,
    "Energy Sold (MWh)": optimal_energy_sold,
    "Energy Charged (MWh)": optimal_energy_charged,
    "Battery SoC (MWh)": optimal_soc
})
results_df = results_df.round(4)
print(results_df)
print("Total Objective Value:", value(problem.objective))


       Price  Energy Sold (MWh)  Energy Charged (MWh)  Battery SoC (MWh)
0    42.1416               45.0                   0.0                5.0
1    38.4571                0.0                   0.0                5.0
2    14.1165                0.0                  50.0               55.0
3    18.8649                0.0                  40.0               95.0
4    24.5399               50.0                   0.0               45.0
5    11.0825                0.0                  50.0               95.0
6    34.5724               50.0                   0.0               45.0
7    27.5017                0.0                  50.0               95.0
8    39.0768                0.0                   0.0               95.0
9    64.8016                0.0                   0.0               95.0
10   85.9560                0.0                   0.0               95.0
11  100.2723                0.0                   0.0               95.0
12  105.4948               50.0                   0

In the following codes, we wanted to investigate the aggregation of the outputs above. 

In [13]:
# Calculating totals
total_energy_sold = sum(optimal_energy_sold)
total_energy_charged = sum(optimal_energy_charged)
final_soc = optimal_soc[-1]  # Final state of charge at the end of the period

# Results
print(f"Total Energy Sold (MWh): {total_energy_sold}")
print(f"Total Energy Charged (MWh): {total_energy_charged}")
print(f"Final Battery SoC (MWh): {final_soc}")

Total Energy Sold (MWh): 960.0
Total Energy Charged (MWh): 910.0
Final Battery SoC (MWh): 0.0
